# Deep Knowledge Tracing (DKT) - Training Notebook

This notebook demonstrates:
1. Creating synthetic student interaction data
2. Training a DKT LSTM model
3. Evaluating model performance (AUC, Brier score)
4. Comparing with Beta-Bernoulli baseline
5. Saving the trained model for deployment


In [ ]:
import sys
sys.path.append('..')

import numpy as np
import torch
import matplotlib.pyplot as plt
from pathlib import Path

from models.dkt import DKT, DKTPredictor, create_synthetic_data, train_dkt
from models.beta_kt import BetaKT
from evaluation import KTEvaluator

print(f"PyTorch version: {torch.__version__}")
print(f"Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")


## 1. Generate Synthetic Data

We'll create synthetic student interaction sequences with varying abilities and learning effects.


In [ ]:
# Parameters
N_STUDENTS = 200
N_QUESTIONS = 25
SEED = 42

# Generate data
data = create_synthetic_data(
    n_students=N_STUDENTS,
    n_questions=N_QUESTIONS,
    seq_len_range=(15, 60),
    seed=SEED
)

print(f"Generated {len(data)} student sequences")
print(f"Example sequence length: {len(data[0]['attempts'])}")
print(f"Example first 5 attempts: {data[0]['attempts'][:5]}")


## 2. Train DKT Model


In [ ]:
# Split data
np.random.shuffle(data)
train_data = data[:140]
test_data = data[140:]

# Train
model = train_dkt(
    data=train_data,
    n_questions=N_QUESTIONS,
    epochs=20,
    batch_size=32,
    lr=0.001,
    device='cpu',
    save_path='../models/dkt_model.pt'
)

print("Training complete! Model saved.")


## 3. Evaluate and Compare

Now evaluate the DKT model and compare with the Beta-Bernoulli baseline.


In [ ]:
# Load and evaluate
predictor = DKTPredictor(model_path='../models/dkt_model.pt', device='cpu')

y_true = []
y_pred = []

for student in test_data:
    attempts = student['attempts']
    for i in range(5, len(attempts)):
        history = attempts[:i]
        next_attempt = attempts[i]
        prob = predictor.predict_next_question(history, next_attempt['question_id'])
        y_true.append(int(next_attempt['correct']))
        y_pred.append(prob)

# Metrics
dkt_metrics = KTEvaluator.evaluate_model(y_true, y_pred)
print(f"DKT AUC: {dkt_metrics['auc']:.4f}")
print(f"DKT Brier: {dkt_metrics['brier_score']:.4f}")
print(f"DKT Accuracy: {dkt_metrics['accuracy']:.4f}")
